### Installation

For these tutorials, we need to install some additional libraries:

In [ ]:
# Skip these if you have already install these libraries
!pip install "nltk"
!pip install "gensim"
!pip install "thefuzz[speedup]"

In [ ]:
import os
from tqdm import tqdm
from thefuzz import fuzz
from gensim.models import Word2Vec

# Resources for English words (these will need to change if language is different):
from nltk.corpus import words, brown
import gensim
import gensim.downloader

In [ ]:
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-50')

# Fuzzy string matching for OCR

A well-known problem in the digital humanities community is OCR (optical character recognition), which introduces varying degrees of error to the digitized text. OCR-produced text can range from gibberish to almost perfect text, often somewhere in between.

This notebook describes **how DeezyMatch could be used for performing fuzzy string matching between OCR queries and a set of candidates**.

In the image, the left side shows the human-corrected version of an OCRed fragment:

![](images/ocr_aligment.png)

XXX

Source: **[HERE THE SOURCE]**

OCR-induced errors are due to misrecognition of characters, such as `h` recognized as `b`, or `c` regotnized as `o`. These transformations between similarly-looking characters are largely generic (e.g. ‘e’ to ‘c’, or ‘B’ to ‘P’), but different typographies or OCR softwares may lead to dataset-specific errors.

### OCR and word embeddings

This issue becomes obvious when we look at the top nearest neighbours of a specific word, using word embeddings trained on digitized data. Word embeddings (such as those trained with [Word2vec](https://en.wikipedia.org/wiki/Word2vec)) are based on the distributional hypothesis, which is summarized "You shall tell a word by the company it keeps" or, in other word, words that are used and occur in the same contexts tend to purport similar meanings (see [distributional semantics](https://en.wikipedia.org/wiki/Distributional_semantics)).

With word embeddings trained on clean text, the top nearest neighbours are words that are semantically similar or almost interchangeable. With word embeddings trained on OCRed text, very often the top nearest neighbours are just OCR variations of the target token, as can be seen in the following example:

![](images/cmp_machineNN.png)

**Source:** `glove-wiki-gigaword-50` embeddings.

![](images/OCR_machineNN.png)

**Source:** word embeddings trained from Living with Machines newspapers, by Nilo Pedrazzini.

## Datasets

To train and use DeezyMatch to find candidates for OCRed queries, we need to prepare the following datasets:
* **String matching dataset:** dataset of string pairs with positive and negative matches (e.g. `maciiine` is  an OCR variation of `machine`, `maciiine` is not an OCR variation of `device`). In the following cells, we show how to build a dataset of string pairs using word embeddings trained on digitized text. 

The idea is the following: 

- if the string similarity between the nearest neighbour(s) and the target word is very high AND if one is not a subset of the other (such as `machine` and `machines`), in most cases, it will be a string variation of the target word (we have tested this empirically); 
- if the string similarity between the nearest neighbour and the target word is low (such as `maciiine` and `device`), it will probably be a synonym or near-synonym, and therefore not a string variation. 

We use this intuition to construct a dataset of positive and negative matches. See some examples for the target word `would`:

  ```
  would	  might     FALSE
  would	  must      FALSE
  would	  likely    FALSE
  would	  thought   FALSE
  would	  woull     TRUE
  would	  wonld     TRUE
  would	  woubl     TRUE
  would	  wouid     TRUE
  ```

* **Candidates dataset:** in this example, list of words in English (obtained from a dictionary and the Brown corpus).
* **Queries dataset:** in this example, list of OCRed words.

We provide the [vocabulary file](https://github.com/Living-with-machines/DeezyMatch/examples/ocr_with_w2v/inputs/characters_v001.vocab) and the [input file](https://github.com/Living-with-machines/DeezyMatch/examples/ocr_with_w2v/inputs/input_dfm.yaml).

### Obtaining the word2vec models

This experiment is with word embeddings trained on digitized British newspapers published between 1860 and 1870. Unfortunately, we can't yet share our embeddings. You can try with other pretrained word embeddings in digitized newspapers in other languages, such as:
* https://openhumanitiesdata.metajnl.com/articles/10.5334/johd.22/
* https://zenodo.org/record/4892800#.YtKldsHMLeo

Download a model and store it as `data/[model_name]/w2v.model`.

### Directory structure

The `ocr_with_w2v` directory should now look similar to this:
```
ocr_with_w2v
   ├── prepare_dataset.ipynb
   ├── tutorial_ocr.ipynb
   ├── data
   │   └── w2v_1860s_news
   │       └── w2v.model
   ├── images
   │   └── ...
   └── inputs
       ├── characters_v001.vocab
       └── input_dfm.yaml
```

## Prepare the string pairs dataset

In [ ]:
# Specify the folder name of the w2v model:
model_name = "w2v_1860s_news"

In [ ]:
# Read the word2vec model
path2model = os.path.join("data", model_name, "w2v.model")
model = Word2Vec.load(path2model)

In [ ]:
# Words in the embeddings:
w2v_words = list(model.wv.index_to_key)

In [ ]:
print(type(glove_vectors))

In [ ]:
# Words in the English language:
nltk_words = set(words.words())
brown_words = set(brown.words())
glove_words = set(glove_vectors.index_to_key)

english_words = nltk_words.union(brown_words).union(glove_words)

In [ ]:
def obtain_matches(word, sims):
    """
    Given a word and the top 100 nearest neighbours, separate into positive and negative matches.
    
    Arguments:
        word (str): a word.
        sims (list): the list of 100 nearest neighbours from the OCR word2vec model.
        
    Returns:
        positive (list): a list of postive matches.
        negative (list): a list of negative matches.
    """
    negative = []
    positive = [word]
    for nn in sims:
        nn_word = nn[0]
        # If one word is not a subset of another:
        if not nn_word in word and not word in nn_word:
            # Split both the word and the nearest neighbour in two parts: the idea is that both
            # parts should be equally similar or equally dissimilar, in order to consider them
            # as positive or negative matches (e.g. "careless" and "listless" are two clear words
            # but have high string similarity due to a big chunk of the word---the suffix---being
            # identical):
            nn_word_1 = nn_word[:len(nn_word)//2]
            nn_word_2 = nn_word[len(nn_word)//2:]
            word_1 = word[:len(word)//2]
            word_2 = word[len(word)//2:]
            # If the nearest neighbour word is a word of the English language
            # and the string similarity is less than 0.50, we consider it a
            # negative match (i.e. not an OCR variation):
            if nn_word in english_words and fuzz.ratio(nn_word_1, word_1) < 50 and fuzz.ratio(nn_word_2, word_2) < 50:
                negative.append(nn_word)
            # If the nearest neighbour word is not a word of the English language
            # and the string similarity is more than 0.50, we consider it a
            # positive match (i.e. an OCR variation):
            if not nn_word in english_words and fuzz.ratio(nn_word_1, word_1) > 50 and fuzz.ratio(nn_word_2, word_2) > 50:
                positive.append(nn_word)
    return positive, negative

In [ ]:
# For each word in the w2v model, keep likely positive and negative matches:
positive_matches = []
negative_matches = []
for word in tqdm(w2v_words):
    # For each word in the w2v model that is longer than 4 characters and 
    # is a word in the English language:
    if len(word) > 4 and word in english_words:
        # Get the top 100 nearest neighbors
        sims = model.wv.most_similar(word, topn=100)
        # Distinguist between positive and negative matches, where
        # * a positive match is an OCR word variation
        # * a negative match is a different word
        positive, negative = obtain_matches(word, sims)
        # We should have the same number of positive matches as negative:
        shortest_length = min([len(positive), len(negative)])
        negative = negative[:shortest_length]
        positive = positive[:shortest_length]
        # Prepare for writing into file:
        negative_matches += [word + "\t" + x + "\t" + "FALSE\n" for x in negative]
        positive_matches += [word + "\t" + x + "\t" + "TRUE\n" for x in positive]

In [ ]:
# Write string pairs into a file:
with open("data/w2v_ocr_pairs_" + model_name + ".txt", "w") as fw:
    for nm in negative_matches:
        fw.write(nm)
    for pm in positive_matches:
        fw.write(pm)

## Prepare the candidates dataset

In [ ]:
with open("data/w2v_ocr_pairs_" + model_name + ".txt") as fr:
    pairs = fr.readlines()

# Write queries into a file (in this case, all words in the OCR word2vec model vocabulary
# if they are in the list of English words):
with open("data/candidates_" + model_name + ".txt", "w") as fw:
    for p in pairs:
        candidate = p.split("\t")
        if candidate[0] in english_words:
            fw.write(candidate[0] + "\n")

## Prepare the queries dataset

In [ ]:
with open("data/w2v_ocr_pairs_" + model_name + ".txt") as fr:
    pairs = fr.readlines()

# Write queries into a file (in this case, all words in the OCR word2vec model vocabulary
# if they are not in the list of English words):
with open("data/queries_" + model_name + ".txt", "w") as fw:
    for p in pairs:
        query = p.split("\t")
        if len(query[1]) > 4:
            if not query[1] in english_words:
                fw.write(query[1] + "\n")